# Models

In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.extended_kalman_vmm import ExtendedKalman, SystemMatrixes
from wPCC_pipeline.pipelines.brix.nodes import initial_parameters, extract_added_masses
from wPCC_pipeline.pipelines.preprocess.nodes import load, add_thrust
from wPCC_pipeline.pipelines.filter_data_extended_kalman.nodes import guess_covariance_matrixes, initial_state, extended_kalman_filter, extended_kalman_smoother
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from phd.pipelines.load_7m.nodes import fix_interpolated_angle
from sklearn.metrics import mean_squared_error
from vessel_manoeuvring_models.models.vmm import VMM, Simulator, ModelSimulator, get_coefficients
import sympy as sp
from functools import reduce
from operator import add
from vessel_manoeuvring_models.symbols import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models.models.propeller import *
from vessel_manoeuvring_models.models.semiempirical_rudder import SemiempiricalRudderSystem
from vessel_manoeuvring_models.models.wind_force import WindForceSystem
from IPython.display import display
from vessel_manoeuvring_models.prime_system import df_prime
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, get_function_subs
from vessel_manoeuvring_models.models.modular_simulator import ModularVesselSimulator
from vessel_manoeuvring_models.models.subsystem import PrimeEquationSubSystem, EquationSubSystem
from vessel_manoeuvring_models.parameters import df_parameters
from phd.helpers import mean
from vessel_manoeuvring_models.angles import mean_angle
from vessel_manoeuvring_models.angles import smallest_signed_angle
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models.modular_simulator import subs_simpler
import statsmodels.api as sm
from wPCC_pipeline.pipelines.vct_data.nodes import vct_scaling
from vct.bokeh_plotting import create_tab
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import inspect
from sympy.printing import pretty
p = df_parameters["symbol"]

import logging
log = logging.getLogger(__name__)

import matplotlib.pyplot as plt
plt.style.use('bmh')

import paper
import model_names

Loading BokehJS ...

[08/15/23 10:33:49] INFO     Loading data from 'wPCC.models' (PartitionedDataSet)...            ]8;id=234776;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=452988;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [3]:
model_names.model_names

,model_name,vmm,regression,model_name_simple,wind
vmm_7m_vct.MDL_hull_inverse_dynamics,vmm_7m_vct.MDL_hull_inverse_dynamics,vmm_7m_vct,MDL_hull_inverse_dynamics,M1.MDL_HID,False
vmm_7m_vct.VCT_MDL_resistance,vmm_7m_vct.VCT_MDL_resistance,vmm_7m_vct,VCT_MDL_resistance,M1.VCT_MDL_resistance,False
vmm_7m_vct.vct,vmm_7m_vct.vct,vmm_7m_vct,vct,M1.vct,False
vmm_7m_vct_wind.MDL_hull_inverse_dynamics,vmm_7m_vct_wind.MDL_hull_inverse_dynamics,vmm_7m_vct_wind,MDL_hull_inverse_dynamics,M2.MDL_HID,True
vmm_7m_vct_wind.VCT_MDL_resistance,vmm_7m_vct_wind.VCT_MDL_resistance,vmm_7m_vct_wind,VCT_MDL_resistance,M2.VCT_MDL_resistance,True
vmm_7m_vct_wind.vct,vmm_7m_vct_wind.vct,vmm_7m_vct_wind,vct,M2.vct,True
vmm_martins_simple.MDL_hull_inverse_dynamics,vmm_martins_simple.MDL_hull_inverse_dynamics,vmm_martins_simple,MDL_hull_inverse_dynamics,M3.MDL_HID,False
vmm_martins_simple.VCT_MDL_resistance,vmm_martins_simple.VCT_MDL_resistance,vmm_martins_simple,VCT_MDL_resistance,M3.VCT_MDL_resistance,False
vmm_martins_simple.vct,vmm_martins_simple.vct,vmm_martins_simple,vct,M3.vct,False
vmm_martins_simple_thrust.MDL_inverse_dynamics,vmm_martins_simple_thrust.MDL_inverse_dynamics,vmm_martins_simple_thrust,MDL_inverse_dynamics,M4.MDL_ID,False


In [4]:
model_names.vmm_simple_names

{'vmm_7m_vct': 'M1',
 'vmm_7m_vct_wind': 'M2',
 'vmm_martins_simple': 'M3',
 'vmm_martins_simple_thrust': 'M4',
 'vmm_simple': 'M5',
 'vmm_simple_wind': 'M6'}

In [5]:
main_model = catalog.load("main_model")

[08/15/23 10:33:50] INFO     Loading data from 'main_model' (PickleDataSet)...                  ]8;id=479565;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=877145;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [6]:
eq_X = main_model.X_eq
eq_X_simple = eq_X.subs(get_function_subs(eq_X))
display(eq_X)
display(eq_X_simple)

Eq(m*(\dot{u} - r**2*x_G - r*v), X_{\dot{u}}*\dot{u} + X_H(u, v, r, delta) + X_P(u, v, r, rev) + X_R(u, v, r, delta, thrust) + X_W(C_x0, C_x3, C_x2, C_x4, aws, C_x5, rho_A, C_x1, awa, A_XV))

Eq(m*(\dot{u} - r**2*x_G - r*v), X_H + X_P + X_R + X_W + X_{\dot{u}}*\dot{u})

In [7]:
eq_Y = main_model.Y_eq
eq_Y_simple = eq_Y.subs(get_function_subs(eq_Y))
display(eq_Y)
display(eq_Y_simple)

Eq(m*(\dot{r}*x_G + \dot{v} + r*u), Y_{\dot{r}}*\dot{r} + Y_{\dot{v}}*\dot{v} + Y_H(u, v, r, delta) + Y_P(u, v, r, rev) + Y_R(u, v, r, delta, thrust) + Y_W(C_y3, aws, rho_A, C_y1, awa, C_y2, A_YV))

Eq(m*(\dot{r}*x_G + \dot{v} + r*u), Y_H + Y_P + Y_R + Y_W + Y_{\dot{r}}*\dot{r} + Y_{\dot{v}}*\dot{v})

In [8]:
eq_N = main_model.N_eq
eq_N_simple = eq_N.subs(get_function_subs(eq_N))
display(eq_N)
display(eq_N_simple)

Eq(I_z*\dot{r} + m*x_G*(\dot{v} + r*u), N_{\dot{r}}*\dot{r} + N_{\dot{v}}*\dot{v} + N_H(u, v, r, delta) + N_P(u, v, r, rev) + N_R(u, v, r, delta, thrust) + N_W(C_n1, C_n2, aws, rho_A, C_n3, L, awa, A_YV))

Eq(I_z*\dot{r} + m*x_G*(\dot{v} + r*u), N_H + N_P + N_R + N_W + N_{\dot{r}}*\dot{r} + N_{\dot{v}}*\dot{v})

In [9]:
paper.save_eq(eq=eq_X_simple, file_name='X')
paper.save_eq(eq=eq_Y_simple, file_name='Y')
paper.save_eq(eq=eq_N_simple, file_name='N')

In [11]:
model = catalog.load("wPCC.vmm_simple")

[08/15/23 10:34:15] INFO     Loading data from 'wPCC.vmm_simple' (PickleDataSet)...             ]8;id=445738;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=542335;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [14]:
hull = model.subsystems['hull']

In [16]:
hull.equations['X_H']

Eq(X_H, X_{0} + X_{u}*u)

In [17]:
hull.equations['Y_H']

Eq(Y_H, Y_{r}*r + Y_{v}*v)

In [18]:
hull.equations['N_H']

Eq(N_H, N_{r}*r + N_{v}*v)

In [20]:
model = catalog.load("wPCC.vmm_martins_simple")

[08/15/23 10:39:25] INFO     Loading data from 'wPCC.vmm_martins_simple' (PickleDataSet)...     ]8;id=528661;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=451164;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [21]:
hull = model.subsystems['hull']

In [22]:
hull.equations['Y_H']

Eq(Y_H, Y_{r}*r + Y_{ur}*r*u + Y_{u}*u + Y_{v}*v)

In [24]:
model_loaders = catalog.load("wPCC.models")

[08/15/23 10:42:19] INFO     Loading data from 'wPCC.models' (PartitionedDataSet)...            ]8;id=389555;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=707666;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [32]:
_ = []
for name, model_loader in model_loaders.items():
    model = model_loader()
    s = pd.Series(model.parameters, name=name)
    _.append(s)
    
df_parameters = pd.DataFrame(_)

In [46]:
df_parameters['model_name'] = df_parameters.index
df_parameters['vmm'] = df_parameters["model_name"].apply(lambda x: x.split(".")[0])
df_parameters["model_name_simple"] = df_parameters.apply(model_names.simple_model_names, axis=1)

In [47]:
df_parameters

,Xudot,Yvdot,Yrdot,Nvdot,Nrdot,C0_w_p0,C1_w_p0,k_0,k_1,k_2,g,Xthrust,C_L_tune,delta_lim,kappa,nu,X0,Xrr,Xu,Xvr,Xvv,Yvrr,Yvvv,Yv,Nrrr,Nr,Nvvr,Nvvv,Nv,C_x0,C_x1,C_x2,C_x3,C_x4,C_x5,C_y1,C_y2,C_y3,C_n1,C_n2,C_n3,Xuu,Yr,Yur,Yu,Nur,Nu,Xdeltadelta,Ydelta,Ythrustdelta,Ythrust,Yvvdelta,Ndelta,Nthrustdelta,Nthrust,Nvvdelta,model_name,vmm,model_name_simple
vmm_7m_vct.MDL_hull_inverse_dynamics,0.000179,-0.006109,-0.000303,-0.000128,-0.000299,0.103786,0.246905,0.576582,-0.368368,-0.07543,9.81,0.883,1.0,1.396263,0.85,0.000001,-0.000953,-0.004460,0.000258,-0.007110,-0.003493,0.029348,-0.114208,-0.005917,0.004665,-0.002219,-0.040354,-0.012335,-0.003579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vmm_7m_vct.MDL_hull_inverse_dynamics,vmm_7m_vct,M1.MDL_HID
vmm_7m_vct.VCT_MDL_resistance,0.000179,-0.006109,-0.000303,-0.000128,-0.000299,0.103786,0.246905,0.576582,-0.368368,-0.07543,9.81,0.883,1.0,1.396263,0.85,0.000001,-0.001035,-0.000346,0.000351,-0.002921,-0.001306,-0.068250,-0.123330,-0.003290,-0.002509,-0.000781,-0.075570,-0.024004,-0.002533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vmm_7m_vct.VCT_MDL_resistance,vmm_7m_vct,M1.VCT_MDL_resistance
vmm_7m_vct.vct,0.000179,-0.006109,-0.000303,-0.000128,-0.000299,0.103786,0.246905,0.576582,-0.368368,-0.07543,9.81,0.883,1.0,1.396263,0.85,0.000001,-0.000334,-0.000346,0.000055,-0.002921,-0.001306,-0.068250,-0.123330,-0.003290,-0.002509,-0.000781,-0.075570,-0.024004,-0.002533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vmm_7m_vct.vct,vmm_7m_vct,M1.vct
vmm_7m_vct_wind.MDL_hull_inverse_dynamics,0.000179,-0.006109,-0.000303,-0.000128,-0.000299,0.103786,0.246905,0.576582,-0.368368,-0.07543,9.81,0.883,1.0,1.396263,0.85,0.000001,-0.000936,-0.004443,0.000260,-0.007065,-0.003474,0.029344,-0.114340,-0.005666,0.004666,-0.002219,-0.040351,-0.012411,-0.003533,-0.29859,-0.45967,2.036783,-2.133305,0.879339,-0.122661,-0.791885,0.297789,-0.013928,-0.15786,0.162606,-0.036097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vmm_7m_vct_wind.MDL_hull_inverse_dynamics,vmm_7m_vct_wind,M2.MDL_HID
vmm_7m_vct_wind.VCT_MDL_resistance,0.000179,-0.006109,-0.000303,-0.000128,-0.000299,0.103786,0.246905,0.576582,-0.368368,-0.07543,9.81,0.883,1.0,1.396263,0.85,0.000001,-0.001035,-0.000346,0.000351,-0.002921,-0.001306,-0.068250,-0.123330,-0.003290,-0.002509,-0.000781,-0.075570,-0.024004,-0.002533,-0.29859,-0.45967,2.036783,-2.133305,0.879339,-0.122661,-0.791885,0.297789,-0.013928,-0.15786,0.162606,-0.036097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vmm_7m_vct_wind.VCT_MDL_resistance,vmm_7m_vct_wind,M2.VCT_MDL_resistance
vmm_7m_vct_wind.vct,0.000179,-0.006109,-0.000303,-0.000128,-0.000299,0.103786,0.246905,0.576582,-0.368368,-0.07543,9.81,0.883,1.0,1.396263,0.85,0.000001,-0.000334,-0.000346,0.000055,-0.002921,-0.001306,-0.068250,-0.123330,-0.003290,-0.002509,-0.000781,-0.075570,-0.024004,-0.002533,-0.29859,-0.45967,2.036783,-2.133305,0.879339,-0.122661,-0.791885,0.297789,-0.013928,-0.15786,0.162606,-0.036097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vmm_7m_vct_wind.vct,vmm_7m_vct_wind,M2.vct
vmm_martins_simple.MDL_hull_inverse_dynamics,0.000179,-0.006109,-0.000303,-0.000128,-0.000299,0.103786,0.246905,0.576582,-0.368368,-0.07543,9.81,0.883,1.0,1.396263,0.85,0.000001,NaN,-0.001552,-0.004166,0.002746,NaN,NaN,NaN,-0.014667,NaN,-0.005375,NaN,NaN,-0.003054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005065,-0.001731,-0.007045,-0.000049,0.007276,-0.000007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vmm_martins_simple.MDL_hull_inverse_dynamics,vmm_martins_simple,M3.MDL_HID
vmm_martins_simple.VCT_MDL_resistance,0.000179,-0.006109,-0.000303,-0.000128,-0.000299,0.103786,0.246905,0.576582,-0.368368,-0.07543,9.81,0.883,1.0,1.396263,0.85,0.000001,NaN,-0.002284,-0.004095,0.001128,NaN,NaN,NaN,-0.006829,NaN,-0.001037

In [35]:
model.subsystems

{'hull': <vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem at 0x7f794f974850>,
 'rudders': <vessel_manoeuvring_models.models.subsystem.PrimeEquationSubSystem at 0x7f794f4bdb10>,
 'propellers': <vessel_manoeuvring_models.models.propeller.PropellersSystem at 0x7f794f4be7d0>,
 'wind_force': <vessel_manoeuvring_models.models.wind_force.WindForceSystem at 0x7f794f4bf7c0>}

In [41]:
df_parameters.apply(model_names.simple_model_names, axis=1)

vmm_7m_vct.MDL_hull_inverse_dynamics                         M1.MDL_HID
vmm_7m_vct.VCT_MDL_resistance                     M1.VCT_MDL_resistance
vmm_7m_vct.vct                                                   M1.vct
vmm_7m_vct_wind.MDL_hull_inverse_dynamics                    M2.MDL_HID
vmm_7m_vct_wind.VCT_MDL_resistance                M2.VCT_MDL_resistance
vmm_7m_vct_wind.vct                                              M2.vct
vmm_martins_simple.MDL_hull_inverse_dynamics                 M3.MDL_HID
vmm_martins_simple.VCT_MDL_resistance             M3.VCT_MDL_resistance
vmm_martins_simple.vct                                           M3.vct
vmm_martins_simple_thrust.MDL_inverse_dynamics                M4.MDL_ID
vmm_simple.VCT_MDL_resistance                     M5.VCT_MDL_resistance
vmm_simple.vct                                                   M5.vct
vmm_simple_wind.VCT_MDL_resistance                M6.VCT_MDL_resistance
vmm_simple_wind.vct                                             

In [42]:
df_parameters["model_name"].apply(lambda x: x.split(".")[0])

vmm_7m_vct.MDL_hull_inverse_dynamics                             vmm_7m_vct
vmm_7m_vct.VCT_MDL_resistance                                    vmm_7m_vct
vmm_7m_vct.vct                                                   vmm_7m_vct
vmm_7m_vct_wind.MDL_hull_inverse_dynamics                   vmm_7m_vct_wind
vmm_7m_vct_wind.VCT_MDL_resistance                          vmm_7m_vct_wind
vmm_7m_vct_wind.vct                                         vmm_7m_vct_wind
vmm_martins_simple.MDL_hull_inverse_dynamics             vmm_martins_simple
vmm_martins_simple.VCT_MDL_resistance                    vmm_martins_simple
vmm_martins_simple.vct                                   vmm_martins_simple
vmm_martins_simple_thrust.MDL_inverse_dynamics    vmm_martins_simple_thrust
vmm_simple.VCT_MDL_resistance                                    vmm_simple
vmm_simple.vct                                                   vmm_simple
vmm_simple_wind.VCT_MDL_resistance                          vmm_simple_wind
vmm_simple_w